In [1]:
import tensorflow as tf
from tensorflow.python.ops import embedding_ops
import numpy as np
tf.compat.v1.disable_eager_execution()

In [2]:
x = tf.compat.v1.placeholder(shape=[10, 6], dtype=tf.float32)
y = tf.compat.v1.placeholder(shape=[10, 1], dtype=tf.float32)
x,y

(<tf.Tensor 'Placeholder:0' shape=(10, 6) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(10, 1) dtype=float32>)

In [3]:
embedding = tf.Variable(tf.random.uniform(shape=[6,1])) # bkt=sparse_length
bias=tf.Variable(tf.zeros([1,1]))
logits = tf.matmul(x, embedding) + bias
logistic = tf.sigmoid(logits)
logits,logistic

(<tf.Tensor 'add:0' shape=(10, 1) dtype=float32>,
 <tf.Tensor 'Sigmoid:0' shape=(10, 1) dtype=float32>)

In [4]:
# loss
loss = tf.reduce_mean(-logits * y + tf.math.log( 1 + tf.math.exp(logits)))
optimizer=tf.compat.v1.train.GradientDescentOptimizer(0.01).minimize(loss)
loss,optimizer

(<tf.Tensor 'Mean:0' shape=() dtype=float32>,
 <tf.Operation 'GradientDescent' type=NoOp>)

In [5]:
x_input = np.reshape(np.random.binomial(1,0.5,size=600),[10,10,6])
y_input = np.reshape(np.random.binomial(1,0.3,size=100),[10,10,1])
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    for epoch in range(2):
      for step in range(10):
        sess.run(optimizer, feed_dict={x:x_input[step], y:y_input[step]})
      print('epoch:{}, loss:{}'.format(epoch, sess.run(loss, feed_dict={x:x_input[step], y:y_input[step]})))
    print(sess.run(logistic, feed_dict={x:x_input[9]}))

epoch:0, loss:0.9692468643188477
epoch:1, loss:0.9311745762825012
[[0.8782885 ]
 [0.8137677 ]
 [0.69507444]
 [0.9218381 ]
 [0.74538004]
 [0.8136641 ]
 [0.89147127]
 [0.6949296 ]
 [0.7277851 ]
 [0.7395273 ]]


## focal_loss

$$focal_loss = $$

In [8]:
gamma = 2
alpha = 1
epsilon = 1e-7
y = tf.constant([1.0,1,0,1])
logits = tf.constant([0.8,0.7,-0.4,0.9])
logistic = tf.sigmoid(logits)
preds = tf.where(tf.equal(y, 1.), logistic, 1. - logistic)
focal_loss = -alpha * (1. - preds) ** gamma * tf.math.log(preds + epsilon)
focal_loss

<tf.Tensor 'mul_4:0' shape=(4,) dtype=float32>

## origin loss

In [13]:
tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))

<tf.Tensor 'Mean_4:0' shape=() dtype=float32>

In [10]:
tf.reduce_mean(logits*(1-y)+tf.math.log(1+tf.math.exp(-logits)))

<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

In [11]:
tf.reduce_mean(-logits * y + tf.math.log( 1 + tf.math.exp(logits))) #公式等价于上

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>